In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\Harsh\Downloads\dirty_cafe_sales.csv")
df.head(20)

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
0,TXN_1961373,Coffee,2,2,4,Credit Card,Takeaway,08-09-2023
1,TXN_4977031,Cake,4,3,12,Cash,In-store,16-05-2023
2,TXN_4271903,Cookie,4,1,ERROR,Credit Card,In-store,19-07-2023
3,TXN_7034554,Salad,2,5,10,UNKNOWN,UNKNOWN,27-04-2023
4,TXN_3160411,Coffee,2,2,4,Digital Wallet,In-store,11-06-2023
5,TXN_2602893,Smoothie,5,4,20,Credit Card,NaN,31-03-2023
6,TXN_4433211,UNKNOWN,3,3,9,ERROR,Takeaway,06-10-2023
7,TXN_6699534,Sandwich,4,4,16,Cash,UNKNOWN,28-10-2023
8,TXN_4717867,NaN,5,3,15,NaN,Takeaway,28-07-2023
9,TXN_2064365,Sandwich,5,4,20,NaN,In-store,31-12-2023


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    10000 non-null  object
 1   item              9667 non-null   object
 2   quantity          9862 non-null   object
 3   price_per_unit    9821 non-null   object
 4   total_spent       9827 non-null   object
 5   payment_method    7421 non-null   object
 6   location          6735 non-null   object
 7   transaction_date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


### We will strip the space from columns and each rows

In [4]:
df = df.apply(lambda x : x.str.strip())
df.columns = df.columns.str.strip()

In [5]:
df.item.unique() # different values are 'UNKNOWN' and 'ERROR'

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'UNKNOWN',
       'Sandwich', nan, 'ERROR', 'Juice', 'Tea'], dtype=object)

### Replce 'UNKNOWN' and 'ERROR' with null

In [6]:
df.replace(['UNKNOWN', 'ERROR', np.nan], pd.NA, inplace = True)

In [7]:
df.isna().sum()

transaction_id         0
item                 969
quantity             479
price_per_unit       533
total_spent          502
payment_method      3178
location            3961
transaction_date     460
dtype: int64

### Let's change the data typs

In [8]:
df[['quantity', 'price_per_unit', 'total_spent']] = df[['quantity', 'price_per_unit', 'total_spent']].apply(pd.to_numeric, errors = 'coerce')
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors = 'coerce', format = '%d-%m-%Y')

In [9]:
df.dtypes

transaction_id              object
item                        object
quantity                   float64
price_per_unit             float64
total_spent                float64
payment_method              object
location                    object
transaction_date    datetime64[ns]
dtype: object

### Let's fill the price_per_unit

In [10]:
df[['item', 'price_per_unit']].drop_duplicates()
df.groupby('item')['price_per_unit'].median()

item
Cake        3.0
Coffee      2.0
Cookie      1.0
Juice       3.0
Salad       5.0
Sandwich    4.0
Smoothie    4.0
Tea         1.5
Name: price_per_unit, dtype: float64

In [11]:
product = [
    (df['item'] == 'Coffee') & df['price_per_unit'].isna(),
    (df['item'] == 'Cookie') & df['price_per_unit'].isna(),
    (df['item'] == 'Salad') & df['price_per_unit'].isna(),
    (df['item'] == 'Tea') & df['price_per_unit'].isna(),
    (df['item'] == 'Cake') & df['price_per_unit'].isna(),
    (df['item'] == 'Juice') & df['price_per_unit'].isna(),
    (df['item'] == 'Sandwich') & df['price_per_unit'].isna(),
    (df['item'] == 'Smoothie') & df['price_per_unit'].isna(),
]
value = (2, 1, 5, 1.5, 3, 3, 4,4)

df['price_per_unit'] = np.select(product, value, default = df['price_per_unit'])

In [12]:
mask = (
    df['quantity'].notna() &
    df['total_spent'].notna() &
    df['price_per_unit'].isna()
)

df.loc[mask, 'price_per_unit'] = df.loc[mask, 'total_spent'] / df.loc[mask, 'quantity']

In [13]:
df['price_per_unit'].isna().sum()

np.int64(6)

### Let's fill the quantity and total spent columns

In [14]:
mask = (
    df['quantity'].isna() &
    df['total_spent'].notna() &
    df['price_per_unit'].notna()
)

df.loc[mask, 'quantity'] = (df.loc[mask, 'total_spent'] / df.loc[mask, 'price_per_unit'])

mask = (
    df['quantity'].notna() &
    df['total_spent'].isna() &
    df['price_per_unit'].notna()
)

df.loc[mask, 'total_spent'] = (df.loc[mask, 'price_per_unit'] * df.loc[mask, 'quantity'])

In [15]:
df['quantity'].isna().sum() 

np.int64(23)

In [16]:
df[df['quantity'].isna()]

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
236,TXN_8562645,Salad,NaN,5.0,NaN,<NA>,In-store,2023-05-18
278,TXN_3229409,Juice,NaN,3.0,NaN,Cash,Takeaway,2023-04-15
641,TXN_2962976,Juice,NaN,3.0,NaN,<NA>,<NA>,2023-03-17
738,TXN_8696094,Sandwich,NaN,4.0,NaN,<NA>,Takeaway,2023-05-14
2796,TXN_9188692,Cake,NaN,3.0,NaN,Credit Card,<NA>,2023-12-01
3203,TXN_4565754,Smoothie,NaN,4.0,NaN,Digital Wallet,Takeaway,2023-10-06
3224,TXN_6297232,Coffee,NaN,2.0,NaN,<NA>,<NA>,2023-04-07
3401,TXN_3251829,Tea,NaN,1.5,NaN,Digital Wallet,In-store,2023-07-25
3779,TXN_7376255,<NA>,NaN,NaN,25.0,<NA>,In-store,2023-05-27
4257,TXN_6470865,Coffee,NaN,2.0,NaN,Digital Wallet,Takeaway,2023-09-18


In [17]:
item_location_relation = df.pivot_table(
    index = 'item',
    columns = 'location',
    values = 'transaction_id',
    aggfunc = 'count',
    fill_value = 0,
    margins = True,
    margins_name = 'Sum'
)
item_location_relation

location,In-store,Takeaway,Sum
item,,,
Cake,321,343,664
Coffee,318,351,669
Cookie,332,362,694
Juice,361,341,702
Salad,371,343,714
Sandwich,370,345,715
Smoothie,322,304,626
Tea,328,336,664
Sum,2723,2725,5448


In [18]:
item_payment_relation = df.pivot_table(
    index = 'item',
    columns = 'payment_method',
    values = 'transaction_id',
    aggfunc = 'count',
    fill_value = 0,
    margins = True,
    margins_name = 'Sum'
)
item_payment_relation

payment_method,Cash,Credit Card,Digital Wallet,Sum
item,,,,
Cake,276,253,246,775
Coffee,244,242,265,751
Cookie,245,253,265,763
Juice,266,276,285,827
Salad,285,279,240,804
Sandwich,253,256,255,764
Smoothie,236,252,255,743
Tea,241,232,254,727
Sum,2046,2043,2065,6154


In [19]:
#### cake and juice relations
#### cake is more in takeaway and cash, credit_card
#### sandwich and smoothie relations
#### sandwich is more in cash and instore

In [20]:
df.item.isna().sum()

np.int64(969)

In [21]:
price = [
    (df['price_per_unit'] == 2 ) & df['item'].isna(),
    (df['price_per_unit'] == 1 ) & df['item'].isna(),
    (df['price_per_unit'] == 5 ) & df['item'].isna(),
    (df['price_per_unit'] == 1.5 ) & df['item'].isna(),
]

items = ('Coffee', 'Cookie', 'Salad', 'Tea',)

df['item'] = np.select(price, items, default = df['item'])

In [22]:
def infer_item(row):
    # if the item is empty them
    if pd.isna(row['item']):

        price = row['price_per_unit']
        location = row['location']
        payment = row['payment_method']

        # Cake vs Juice
        if price == 3:
            if (
                not pd.isna(location) and
                not pd.isna(payment) and
                location == 'Takeaway' and
                payment in ['Cash', 'Credit Card']
            ):
                return 'Cake'
            else:
                return 'Juice'

        elif price == 4:
            if (
                not pd.isna(location) and
                not pd.isna(payment) and
                location == 'In-store' and
                payment == 'Cash'
            ):
                return 'Sandwich'
            else:
                return 'Smoothie'

        else:
            return 
            

    # if the item is not empty keep the data same
    return row['item']

In [23]:
df['item'] = df.apply(infer_item, axis = 1)

In [24]:
df.dropna(subset=['quantity', 'item'], inplace=True)

In [25]:
df.isna().sum()

transaction_id         0
item                   0
quantity               0
price_per_unit         0
total_spent            0
payment_method      3168
location            3952
transaction_date     460
dtype: int64

In [26]:
df.sort_values(by = 'transaction_id', inplace = True)

In [27]:
df.head()

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
2987,TXN_1000555,Tea,1.0,1.5,2.0,Credit Card,In-store,2023-10-19
115,TXN_1001832,Salad,2.0,5.0,10.0,Cash,Takeaway,NaT
418,TXN_1002457,Cookie,5.0,1.0,5.0,Digital Wallet,Takeaway,2023-09-29
3417,TXN_1003246,Juice,2.0,3.0,6.0,<NA>,<NA>,2023-02-15
4366,TXN_1004184,Smoothie,1.0,4.0,4.0,Credit Card,In-store,2023-05-18


### Let's fill the missing values from payment_method, location, transaction_date

In [28]:
df['transaction_date'] = df['transaction_date'].ffill()

In [29]:
df.fillna({'payment_method':df['payment_method'].mode()[0],
            'location': df['location'].mode()[0]}, inplace = True)

In [30]:
df.isna().sum()

transaction_id      0
item                0
quantity            0
price_per_unit      0
total_spent         0
payment_method      0
location            0
transaction_date    0
dtype: int64

In [31]:
df.head(20)

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
2987,TXN_1000555,Tea,1.0,1.5,2.0,Credit Card,In-store,2023-10-19
115,TXN_1001832,Salad,2.0,5.0,10.0,Cash,Takeaway,2023-10-19
418,TXN_1002457,Cookie,5.0,1.0,5.0,Digital Wallet,Takeaway,2023-09-29
3417,TXN_1003246,Juice,2.0,3.0,6.0,Digital Wallet,Takeaway,2023-02-15
4366,TXN_1004184,Smoothie,1.0,4.0,4.0,Credit Card,In-store,2023-05-18
3995,TXN_1004563,Tea,5.0,1.5,8.0,Credit Card,In-store,2023-10-28
1886,TXN_1005331,Coffee,1.0,2.0,2.0,Digital Wallet,Takeaway,2023-11-04
8071,TXN_1005377,Cake,5.0,3.0,15.0,Digital Wallet,Takeaway,2023-06-03
9311,TXN_1005472,Coffee,4.0,2.0,8.0,Credit Card,Takeaway,2023-04-21
4744,TXN_1006942,Salad,1.0,5.0,5.0,Credit Card,In-store,2023-11-30


In [32]:
df.tail(10)

,transaction_id,item,quantity,price_per_unit,total_spent,payment_method,location,transaction_date
9141,TXN_9989719,Smoothie,2.0,4.0,8.0,Credit Card,Takeaway,2023-11-26
1244,TXN_9991225,Cake,5.0,3.0,15.0,Credit Card,Takeaway,2023-02-03
1961,TXN_9991468,Juice,4.0,3.0,12.0,Credit Card,Takeaway,2023-11-30
4676,TXN_9992148,Tea,3.0,1.5,5.0,Cash,Takeaway,2023-01-14
2496,TXN_9996195,Juice,5.0,3.0,15.0,Digital Wallet,In-store,2023-03-16
1389,TXN_9996968,Sandwich,1.0,4.0,4.0,Cash,In-store,2023-10-14
7964,TXN_9997263,Cookie,2.0,1.0,2.0,Digital Wallet,Takeaway,2023-03-05
4304,TXN_9997620,Cookie,2.0,1.0,2.0,Cash,Takeaway,2023-12-11
98,TXN_9999113,Juice,4.0,3.0,12.0,Cash,Takeaway,2023-05-27
5037,TXN_9999124,Juice,2.0,3.0,6.0,Digital Wallet,Takeaway,2023-05-27


In [33]:
df.to_csv('Cleaned_Cafe_Dataset.csv', index = False)